In [326]:
import numpy as np

def cosine_similarity_method(input):
  sim_matrix = []

  for x in range(len(input)):
    for y in range(len(input[x])):
      kosong = []

      v1 = np.array(input[x])
      v2 = np.array(input[y])

      for z in range(len(v1)):
        if np.isnan(v1[z]) or np.isnan(v2[z]):
          kosong.append(z)

      v1 = np.delete(v1, kosong)
      v2 = np.delete(v2, kosong)

      v1 = v1.astype(np.float)
      v2 = v2.astype(np.float)

      norma = (sum(v1**2))**(0.5)
      normb = (sum(v2**2))**(0.5)

      atas = np.dot(v1, v2)
      bawah = (sum(v1**2) * sum(v2**2))**(0.5)

      sim = np.dot(v1, v2) / (sum(v1**2) * sum(v2**2))**(0.5)

      print('Sim(i{}, i{}) = "{} * {} divided by {} * {} = {} divided by {} = {}. {} \n'.format(x,y, v1,v2, norma, normb, atas, bawah, sim, kosong))

      if y == 0:
        sim_matrix.append([sim])
      else :
        sim_matrix[x].append(sim)

  return np.array(sim_matrix)


def adjusted_cosine_similarity_method(input):
  # Cari average rating dari user
  input_transpose = np.array(input).transpose()
  print('Inputan User:\n', str(input_transpose))

  average_rating = []

  for x in range(len(input_transpose)):
    v_rating_user = []
    for y in range(len(input_transpose[x])):
      rating_user = input_transpose[x][y]
      if rating_user > 0:
        v_rating_user.append(rating_user)
    
    average_rating.append(sum(v_rating_user) / len(v_rating_user))
  
  print("User Average Rating : ", str(average_rating))

  # Cari Mean Adjusted Rating Matrix 
  mean_adjusted_rating_matrix = []
  for x in range(len(input_transpose)):
    for y in range(len(input_transpose[x])):
      rating_user = input_transpose[x][y]
      mean_rating_user = average_rating[x]
      
      if rating_user > 0:
        mean = rating_user - mean_rating_user
      else:
        mean = np.NaN

      if y == 0:
        mean_adjusted_rating_matrix.append([mean])
      else :
        mean_adjusted_rating_matrix[x].append(mean)

  mean_adjusted_rating_matrix = np.array(mean_adjusted_rating_matrix)
  print("Mean Adjusted Rating Matrix :\n",str(mean_adjusted_rating_matrix))


  # Cosine similarity menerima input rating user terhadap item
  similarity = cosine_similarity_method(mean_adjusted_rating_matrix.transpose())
  return similarity


class RecomenderSystem:
  def __init__(self, input):

    for x in range(len(input)):
      for y in range(len(input[x])):
        if input[x][y] == 0:
          input[x][y] = np.NaN

    self.input = input
    print(np.array(input))

  


  def predict(self, user, item, similarity_function, treshold):
    input = self.input
    user = user - 1
    item = item -1
    sim = []

    if similarity_function == 'cosine similarity' :
      sim = cosine_similarity_method(input)      
    elif similarity_function == 'adjusted cosine similarity':
      sim = adjusted_cosine_similarity_method(input)

    print("Similarity Matrix : \n", str(sim))

    user_ratings = [rate[user] for rate in input]
    item_similarity = sim[item]

    user_ratings = np.delete(user_ratings, item)
    item_similarity = np.delete(item_similarity, item)

    print("\nv1 : ", str(user_ratings), ",\tv2 : ", str(item_similarity))

    print("\nDot Product v1 dan v2 dengan treshold: ", str(treshold))
    dot_user_ratings_and_item_similarity = 0

    for x in range(len(item_similarity)):
      if item_similarity[x] >= treshold:
        dot_user_ratings_and_item_similarity += user_ratings[x] * item_similarity[x]
        print(str(user_ratings[x]), " * ", str(item_similarity[x]), " = ", str(user_ratings[x] * item_similarity[x]))

    pred_rating = dot_user_ratings_and_item_similarity / sum([x for x in item_similarity if x >= treshold])
    print("\nPrediksi Rating", str(round(pred_rating, 1)))   
             

In [331]:
inputan_user = [[5,3,4,3,1],
                [3,1,3,3,5],
                [4,2,4,1,5],
                [4,3,3,5,2],
                [0,3,5,4,1]]
recsys = RecomenderSystem(inputan_user)
recsys.predict(user=1, item=5, similarity_function="adjusted cosine similarity", treshold=0)

[[ 5.  3.  4.  3.  1.]
 [ 3.  1.  3.  3.  5.]
 [ 4.  2.  4.  1.  5.]
 [ 4.  3.  3.  5.  2.]
 [nan  3.  5.  4.  1.]]
Inputan User:
 [[ 5.  3.  4.  4. nan]
 [ 3.  1.  2.  3.  3.]
 [ 4.  3.  4.  3.  5.]
 [ 3.  3.  1.  5.  4.]
 [ 1.  5.  5.  2.  1.]]
User Average Rating :  [4.0, 2.4, 3.8, 3.2, 2.8]
Mean Adjusted Rating Matrix :
 [[ 1.  -1.   0.   0.   nan]
 [ 0.6 -1.4 -0.4  0.6  0.6]
 [ 0.2 -0.8  0.2 -0.8  1.2]
 [-0.2 -0.2 -2.2  1.8  0.8]
 [-1.8  2.2  2.2 -0.8 -1.8]]
Sim(i0, i0) = "[ 1.   0.6  0.2 -0.2 -1.8] * [ 1.   0.6  0.2 -0.2 -1.8] divided by 2.1633307652783933 * 2.1633307652783933 = 4.68 divided by 4.68 = 1.0. [] 

Sim(i0, i1) = "[ 1.   0.6  0.2 -0.2 -1.8] * [-1.  -1.4 -0.8 -0.2  2.2] divided by 2.1633307652783933 * 2.9120439557122073 = -5.92 divided by 6.29971427923521 = -0.9397251585699998. [] 

Sim(i0, i2) = "[ 1.   0.6  0.2 -0.2 -1.8] * [ 0.  -0.4  0.2 -2.2  2.2] divided by 2.1633307652783933 * 3.1432467291003428 = -3.7199999999999993 divided by 6.7998823519234515 = -0.5470682884

Inputan User:
 [[ 5.  3.  4.  4. nan]
 [ 3.  1.  2.  3.  3.]
 [ 4.  3.  4.  3.  5.]
 [ 3.  3.  1.  5.  4.]
 [ 1.  5.  5.  2.  1.]]
User Average Rating :  [4.0, 2.4, 3.8, 3.2, 2.8]
Mean Adjusted Rating Matrix :
 [[ 1.  -1.   0.   0.   nan]
 [ 0.6 -1.4 -0.4  0.6  0.6]
 [ 0.2 -0.8  0.2 -0.8  1.2]
 [-0.2 -0.2 -2.2  1.8  0.8]
 [-1.8  2.2  2.2 -0.8 -1.8]]
Sim(i0, i0) = "[ 1.   0.6  0.2 -0.2 -1.8] * [ 1.   0.6  0.2 -0.2 -1.8] divided by 2.1633307652783933 * 2.1633307652783933 = 4.68 divided by 4.68 = 1.0. [] 

Sim(i0, i1) = "[ 1.   0.6  0.2 -0.2 -1.8] * [-1.  -1.4 -0.8 -0.2  2.2] divided by 2.1633307652783933 * 2.9120439557122073 = -5.92 divided by 6.29971427923521 = -0.9397251585699998. [] 

Sim(i0, i2) = "[ 1.   0.6  0.2 -0.2 -1.8] * [ 0.  -0.4  0.2 -2.2  2.2] divided by 2.1633307652783933 * 3.1432467291003428 = -3.7199999999999993 divided by 6.7998823519234515 = -0.5470682884605702. [] 

Sim(i0, i3) = "[ 1.   0.6  0.2 -0.2 -1.8] * [ 0.   0.6 -0.8  1.8 -0.8] divided by 2.1633307652783933 * 

In [332]:
recsys = RecomenderSystem(inputan_user)
recsys.predict(user=1, item=5, similarity_function="cosine similarity", treshold=0.7)

[[ 5.  3.  4.  3.  1.]
 [ 3.  1.  3.  3.  5.]
 [ 4.  2.  4.  1.  5.]
 [ 4.  3.  3.  5.  2.]
 [nan  3.  5.  4.  1.]]
Sim(i0, i0) = "[5. 3. 4. 3. 1.] * [5. 3. 4. 3. 1.] divided by 7.745966692414834 * 7.745966692414834 = 60.0 divided by 60.0 = 1.0. [] 

Sim(i0, i1) = "[5. 3. 4. 3. 1.] * [3. 1. 3. 3. 5.] divided by 7.745966692414834 * 7.280109889280518 = 44.0 divided by 56.39148871948674 = 0.7802595923450996. [] 

Sim(i0, i2) = "[5. 3. 4. 3. 1.] * [4. 2. 4. 1. 5.] divided by 7.745966692414834 * 7.874007874011811 = 50.0 divided by 60.991802727907626 = 0.8197822947299412. [] 

Sim(i0, i3) = "[5. 3. 4. 3. 1.] * [4. 3. 3. 5. 2.] divided by 7.745966692414834 * 7.937253933193772 = 58.0 divided by 61.48170459575759 = 0.9433700705169153. [] 

Sim(i0, i4) = "[3. 4. 3. 1.] * [3. 5. 4. 1.] divided by 5.916079783099616 * 7.14142842854285 = 42.0 divided by 42.24926034855522 = 0.9941002434954168. [0] 

Sim(i1, i0) = "[3. 1. 3. 3. 5.] * [5. 3. 4. 3. 1.] divided by 7.280109889280518 * 7.745966692414834 = 